<a href="https://colab.research.google.com/github/Nadil-K/DataStorm4.0/blob/main/DataStorm4_0_SemiFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Loading packages
import numpy as np
import pandas as pd

In [ ]:
## Loading dataset
## Change the path to files
freezer_data = pd.read_csv('/content/freezer_data.csv')
outlet_data = pd.read_csv('/content/outlets_data.csv')
product_data = pd.read_csv('/content/product_data.csv')
sales_data = pd.read_csv('/content/sales_data.csv')
week_data = pd.read_csv('/content/week_data.csv')

# Merge sales data with product data
# ignore volume and product name
temp_df = pd.merge(product_data[['pid','price']], sales_data, on = ['pid'], how = 'right')

# merge outlet data with new data 
x_df = pd.merge(temp_df, outlet_data, on = ['Outlet_ID'], how = 'left')

#product type

x_df['product_type'] = x_df['pid'].str[0]
x_df['product_type'] = x_df['product_type'].map({'I':0, 'B':1})


x_df = x_df.drop('product_name',axis=1)


#bins
bins = [0, 4, 7, 10, 13]
labels = ['jan', 'feb', 'mar', 'apr']

x_df['month'] = pd.cut(x_df['week'], bins, labels=labels)

x_df = x_df.drop('week',axis=1)

x_df


#creating Columns
temp = pd.DataFrame()

temp = x_df[['Outlet_ID', 'no_units', 'month', 'product_type']]

grouped_data = temp.groupby(['Outlet_ID', 'month', 'product_type'])['no_units'].sum().reset_index()
grouped_data['product_type'] = grouped_data['product_type'].astype(str)
grouped_data['sale_per_month_by_type'] = grouped_data['month'].str.cat(grouped_data['product_type'], sep='_type_')
grouped_data


#Polishup
pivot_df = grouped_data.pivot_table(index='Outlet_ID', columns='sale_per_month_by_type', values='no_units')

# Reset column index and convert to DataFrame
pivot_df = pivot_df.reset_index().rename_axis(None, axis=1)
pivot_df

In [ ]:
print(outlet_data.duplicated().sum())
print(sales_data.duplicated().sum())
sales_data = sales_data.drop_duplicates()
outlet_data = outlet_data.drop_duplicates(subset='Outlet_ID')


0
0


# Merging Data

In [ ]:
# Merge sales data with product data
# ignore volume and product name
temp_df = pd.merge(product_data[['pid','price']], sales_data, on = ['pid'], how = 'right')

# merge outlet data with new data 
x_df = pd.merge(temp_df, outlet_data, on = ['Outlet_ID'], how = 'left')

x_df

,pid,price,Outlet_ID,week,product_name,no_units,area(sqft)
0,IP1,100,ID7203,1,Vanilla Mini Cone,874.0,5660
1,IP2,90,ID7203,1,Chocolate Petite Bar,105.0,5660
2,IP3,110,ID7203,1,Strawberry Tiny Cup,1198.0,5660
3,IP4,100,ID7203,1,Mint Bite-size Pop,502.0,5660
4,IP5,100,ID7203,1,Butter Pecan Small Stick,710.0,5660
...,...,...,...,...,...,...,...
87383,BP1,1200,ID8207,12,Chocolate Fudge Big Scoop,56.0,3539
87384,BP2,1100,ID8207,12,Moose Tracks Jumbo Cup,60.0,3539
87385,BP3,1150,ID8207,12,Mint Chocolate Chip Family Pack,51.0,3539
87386,BP4,1000,ID8207,12,Rocky Road Mega Tub,78.0,3539


# Create Column for Product Type

In [ ]:
x_df = x_df.drop('product_name_y',axis=1)
x_df = x_df.drop('product_name_x',axis=1)
x_df = x_df.drop('volume',axis=1)

In [ ]:
# x_df['product_volume'] = x_df['product_type'] * x_df['no_units']
# x_df['product_sale'] = x_df['no_units'] * x_df['price']
# x_df['sale_per_volume'] = x_df['product_sale'] / x_df['product_volume']

x_df 



,pid,price,Outlet_ID,week,no_units,area(sqft),product_type
0,IP1,100,ID7203,1,874.0,5660,0
1,IP2,90,ID7203,1,105.0,5660,0
2,IP3,110,ID7203,1,1198.0,5660,0
3,IP4,100,ID7203,1,502.0,5660,0
4,IP5,100,ID7203,1,710.0,5660,0
...,...,...,...,...,...,...,...
87383,BP1,1200,ID8207,12,56.0,3539,1
87384,BP2,1100,ID8207,12,60.0,3539,1
87385,BP3,1150,ID8207,12,51.0,3539,1
87386,BP4,1000,ID8207,12,78.0,3539,1


## Create bins for months

,pid,price,Outlet_ID,no_units,area(sqft),product_type,month
0,IP1,100,ID7203,874.0,5660,0,jan
1,IP2,90,ID7203,105.0,5660,0,jan
2,IP3,110,ID7203,1198.0,5660,0,jan
3,IP4,100,ID7203,502.0,5660,0,jan
4,IP5,100,ID7203,710.0,5660,0,jan
...,...,...,...,...,...,...,...
87383,BP1,1200,ID8207,56.0,3539,1,apr
87384,BP2,1100,ID8207,60.0,3539,1,apr
87385,BP3,1150,ID8207,51.0,3539,1,apr
87386,BP4,1000,ID8207,78.0,3539,1,apr


Creating column for sale_per_month_by_type

,Outlet_ID,month,product_type,no_units,sale_per_month_by_type
0,ID1008,jan,0,13564.0,jan_type_0
1,ID1008,jan,1,0.0,jan_type_1
2,ID1008,feb,0,5763.0,feb_type_0
3,ID1008,feb,1,0.0,feb_type_1
4,ID1008,mar,0,7510.0,mar_type_0
...,...,...,...,...,...
7603,ID9980,feb,1,4856.0,feb_type_1
7604,ID9980,mar,0,20548.0,mar_type_0
7605,ID9980,mar,1,5546.0,mar_type_1
7606,ID9980,apr,0,11490.0,apr_type_0


In [ ]:
columns_to_drop = ['month', 'product_type']
grouped_data = grouped_data.drop(columns=columns_to_drop)
grouped_data

,Outlet_ID,no_units,sale_per_month_by_type
0,ID1008,13564.0,jan_type_0
1,ID1008,0.0,jan_type_1
2,ID1008,5763.0,feb_type_0
3,ID1008,0.0,feb_type_1
4,ID1008,7510.0,mar_type_0
...,...,...,...
7603,ID9980,4856.0,feb_type_1
7604,ID9980,20548.0,mar_type_0
7605,ID9980,5546.0,mar_type_1
7606,ID9980,11490.0,apr_type_0


,Outlet_ID,apr_type_0,apr_type_1,feb_type_0,feb_type_1,jan_type_0,jan_type_1,mar_type_0,mar_type_1
0,ID1008,6165.0,0.0,5763.0,0.0,13564.0,0.0,7510.0,0.0
1,ID1012,0.0,472.0,0.0,869.0,0.0,1053.0,0.0,881.0
2,ID1024,0.0,1230.0,0.0,1644.0,0.0,2188.0,0.0,1881.0
3,ID1029,8135.0,1054.0,10266.0,1670.0,13190.0,3227.0,9189.0,2411.0
4,ID1036,3942.0,592.0,5291.0,704.0,10798.0,1080.0,8172.0,785.0
...,...,...,...,...,...,...,...,...,...
946,ID9960,8585.0,1426.0,9058.0,2072.0,12807.0,3252.0,8651.0,2517.0
947,ID9971,5842.0,0.0,9170.0,0.0,11912.0,0.0,9559.0,0.0
948,ID9973,5666.0,1450.0,7516.0,2143.0,11932.0,2819.0,14385.0,2157.0
949,ID9979,5718.0,0.0,5462.0,0.0,10851.0,0.0,9861.0,0.0


In [ ]:
grouped_data

,Outlet_ID,month,sale,no_units
0,ID1008,1,1365360.0,13564.0
1,ID1008,2,582050.0,5763.0
2,ID1008,3,744060.0,7510.0
3,ID1008,4,599520.0,6165.0
4,ID1012,1,1162350.0,1053.0
...,...,...,...,...
3799,ID9979,4,559100.0,5718.0
3800,ID9980,1,10645000.0,28688.0
3801,ID9980,2,7540240.0,26792.0
3802,ID9980,3,8288360.0,26094.0


In [ ]:
final_df = pd.DataFrame()
final_df = outlet_data

final_df

,Outlet_ID,area(sqft)
0,ID7203,5660
1,ID1878,4138
2,ID1740,5546
3,ID4366,5134
4,ID8358,4100
...,...,...
983,ID4119,3858
984,ID2978,3495
985,ID6549,1141
986,ID1210,3322


In [ ]:
x_df['sale'] = x_df['no_units'] * x_df['price']

In [ ]:
x_df

,pid,price,Outlet_ID,week,no_units,area(sqft),product_type,sale
0,IP1,100,ID7203,1,874.0,5660,0,87400.0
1,IP2,90,ID7203,1,105.0,5660,0,9450.0
2,IP3,110,ID7203,1,1198.0,5660,0,131780.0
3,IP4,100,ID7203,1,502.0,5660,0,50200.0
4,IP5,100,ID7203,1,710.0,5660,0,71000.0
...,...,...,...,...,...,...,...,...
87369,BP1,1200,ID8207,12,56.0,3539,1,67200.0
87370,BP2,1100,ID8207,12,60.0,3539,1,66000.0
87371,BP3,1150,ID8207,12,51.0,3539,1,58650.0
87372,BP4,1000,ID8207,12,78.0,3539,1,78000.0


In [ ]:
x_df = x_df.drop('price',axis=1)
x_df = x_df.drop('no_units',axis=1)

In [ ]:
x_df

,pid,Outlet_ID,week,area(sqft),sale,type
0,IP1,ID7203,1,5660,87400.0,1
1,IP1,ID7203,2,5660,64400.0,1
2,IP1,ID7203,3,5660,60000.0,1
3,IP1,ID7203,4,5660,10600.0,1
4,IP1,ID7203,5,5660,77500.0,1
...,...,...,...,...,...,...
87383,BP5,ID9469,8,1783,93500.0,1
87384,BP5,ID9469,9,1783,56100.0,1
87385,BP5,ID9469,10,1783,80300.0,1
87386,BP5,ID9469,11,1783,69300.0,1


In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import pandas as pd

# Load or create your dataset
df = pd.read_csv('data.csv')

# Select the features you want to use for clustering
X = df[['feature1', 'feature2', 'feature3']]

# Instantiate a clustering algorithm (e.g., K-means)
kmeans = KMeans(n_clusters=3)

# Fit the clustering algorithm to your data
kmeans.fit(X)

# Get the cluster labels for each sample
labels = kmeans.labels_

# Calculate the Silhouette Coefficient
silhouette_coefficient = silhouette_score(X, labels)

# Display the Silhouette Coefficient
print("Silhouette Coefficient:", silhouette_coefficient)


In [ ]:
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import KMeans
import pandas as pd

# Load or create your dataset
df = pd.read_csv('data.csv')

# Select the features you want to use for clustering
X = df[['feature1', 'feature2', 'feature3']]

# Instantiate a clustering algorithm (e.g., K-means)
kmeans = KMeans(n_clusters=3)

# Fit the clustering algorithm to your data
kmeans.fit(X)

# Get the cluster labels for each sample
labels = kmeans.labels_

# Calculate the Davies-Bouldin Index
db_index = davies_bouldin_score(X, labels)



In [ ]:
from sklearn.metrics import calinski_harabasz_score
from sklearn.cluster import KMeans
import pandas as pd

# Load or create your dataset
df = pd.read_csv('data.csv')

# Select the features you want to use for clustering
X = df[['feature1', 'feature2', 'feature3']]

# Instantiate a clustering algorithm (e.g., K-means)
kmeans = KMeans(n_clusters=3)

# Fit the clustering algorithm to your data
kmeans.fit(X)

# Get the cluster labels for each sample
labels = kmeans.labels_

# Calculate the Calinski-Harabasz Index
ch_index = calinski_harabasz_score(X, labels)

# Display the Calinski-Harabasz Index
print("Calinski-Harabasz Index:", ch_index)
